## Import statements and globals

In [95]:
from pulp import *
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift, estimate_bandwidth
from scipy import optimize


HitterPositions = ['C', '1B', '2B', 'SS', '3B', 'OF', 'outer', 'inner', 'util']
HitterMetrics = ['R', 'HR', 'RBI', 'SB', 'AVG']

PitcherPositions = ['SP', 'RP']
PitcherMetrics = ['W', 'SV', 'K', 'ERA', 'WHIP']

## Import data

In [108]:
def get_hitters():
    hitters = pd.read_csv('razzball-hitters.csv', index_col='#', usecols=['#','Name','Team','ESPN','R','HR', 'RBI', 'SB','AVG',])
    hitters.rename_axis('Razzball_Rank', inplace=True)
    hitters.reset_index(inplace=True)

    # sort and rank
    for metric in HitterMetrics:
        hitters.sort_values(by=[metric],inplace=True, ascending=False)
        hitters.reset_index(inplace=True, drop=True)
        hitters.index.rename('{} rank'.format(metric), inplace=True)
        hitters.reset_index(inplace=True)
    hitters['Ovr'] = (hitters['AVG rank'] + hitters['SB rank'] + hitters['RBI rank'] + hitters['HR rank'] + hitters['R rank']) / 5
    #hitters['Ovr'] = (hitters['Ovr'] + hitters['Razzball_Rank']) / 2
    hitters.rename(columns={'ESPN':'POS'}, inplace=True)
    
    hitters = hitters.assign(POS=hitters.POS.str.split('/')).explode('POS')
    hitters.sort_values(by=['Ovr'],inplace=True,ascending=True)
    return hitters

def get_pitchers():
    pitchers = pd.read_csv('razzball-pitchers.csv', index_col='#', usecols=['#','Name','Team','POS','W', 'SV', 'K', 'ERA', 'WHIP'])
    pitchers.rename_axis('Razzball_Rank', inplace=True)
    pitchers.reset_index(inplace=True)
    
    pitchers = pitchers.assign(POS=pitchers.POS.str.split('/')).explode('POS')
    sp = pitchers[pitchers['POS'] == 'SP'].reset_index(drop=True)
    rp = pitchers[pitchers['POS'] == 'RP'].reset_index(drop=True)
    
    for metric in PitcherMetrics:
        if(metric != 'SV'):
            sp.sort_values(by=[metric],inplace=True, ascending=(metric=='WHIP' or metric=='ERA'))
            sp.reset_index(inplace=True, drop=True)
            sp.rename_axis('{} rank'.format(metric), inplace=True)
            sp.reset_index(inplace=True)
        
        if(metric != 'W'):
            rp.sort_values(by=[metric],inplace=True, ascending=(metric=='WHIP' or metric=='ERA'))
            rp.reset_index(inplace=True, drop=True)
            rp.rename_axis('{} rank'.format(metric), inplace=True)
            rp.reset_index(inplace=True)
    
    sp['Ovr'] = (sp['W rank'] + sp['K rank'] + sp['ERA rank'] + sp['WHIP rank']) / 4
    sp.sort_values(by=['Ovr'],inplace=True,ascending=True)
    
    rp['Ovr'] = (rp['SV rank'] + rp['K rank'] + rp['ERA rank'] + rp['WHIP rank']) / 4
    rp.sort_values(by=['Ovr'],inplace=True,ascending=True)
    
    return sp.append(rp).sort_values(by=['Ovr'],ascending=True)
    
def get_hitter_prices(hitters):
    prices = pd.read_csv('razzball-hitters-prices.csv', index_col='#', usecols=['#', 'Name', 'Team', '5×5 $', '$R', '$HR', '$RBI', '$SB', '$AVG (no OBP)'])
    prices.rename(columns={'5×5 $': '$'},inplace=True)
    hitters = hitters.merge(prices, left_on=['Name', 'Team'], right_on=['Name','Team'], how='left')
    return hitters

def get_pitcher_prices(pitchers):
    prices = pd.read_csv('razzball-pitchers-prices.csv', index_col='#', usecols=['#','Name','Team','5×5 $','$W (no QS)','$SV (no HLD)','$K','$WHIP','$ERA'])
    prices.rename(columns={'5×5 $': '$'},inplace=True)
    pitchers = pitchers.merge(prices, left_on=['Name', 'Team'], right_on=['Name','Team'], how='left')
    return pitchers
    
def split_by_pos(hitters, pos):
    if (pos == 'C'):
        return hitters[hitters['POS'] == 'C']
    elif (pos == '1B'):
        return hitters[hitters['POS'] == '1B']
    elif (pos == '2B'):
        return hitters[hitters['POS'] == '2B']
    elif (pos == 'SS'):
        return hitters[hitters['POS'] == 'SS']
    elif (pos == '3B'):
        return hitters[hitters['POS'] == '3B']
    elif (pos == 'OF'):
        return hitters[hitters['POS'] == 'OF']
    elif (pos == 'corner'):
        return hitters[hitters['POS'] == '1B'].append(hitters[hitters['POS'] == '3B'])
    elif (pos == 'inner'):
        return hitters[hitters['POS'] == '2B'].append(hitters[hitters['POS'] == 'SS'])
    elif (pos == 'SP'):
        return hitters[hitters['POS'] == 'SP']
    elif (pos == 'RP'):
        return hitters[hitters['POS'] == 'RP']
    else: # util
        return hitters
        

## Run calculations

In [3]:
def group_players(players):
    bandwidth = estimate_bandwidth(players, quantile=0.05, n_samples=100)
    ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
    labels = ms.fit_predict(players)
    #labels = ms.labels_
    cluster_centers = ms.cluster_centers_

    #labels_unique = np.unique(labels)
    #n_clusters_ = len(labels_unique)

    #print("number of estimated clusters : %d" % n_clusters_)
    players['group'] = labels
    return players

In [4]:
def group_pitchers_pos(pitchers):
    return [group_players(split_by_pos(pitchers, x).drop(['W rank', 'SV rank', 'K rank', 'ERA rank', 'WHIP rank', 'Razzball_Rank','Team','POS'], axis=1)) for x in PitcherPositions]

def group_by_pos(hitters):
    return [group_players(split_by_pos(hitters, x).drop(['AVG rank', 'SB rank', 'RBI rank', 'HR rank', 'R rank', 'Razzball_Rank','Team','POS'], axis=1)) for x in HitterPositions]

### Prep dataframes

In [5]:
hitter_tables = { 'C': "",
                  '1B': "",
                  '2B': "",
                  'SS': "",
                  '3B': "",
                  'inner': "",
                  'outer': "",
                  'OF': "",
                  'util': "" }

pitcher_tables = { 'SP': "",
                   'RP': "" }

In [90]:
hitters = get_hitters().set_index('Name')
groups = []
#C, 1B, 2B, SS, 3B, OF, outer, inner, util
#groups = group_by_pos(hitters)
groups = [split_by_pos(hitters, x) for x in HitterPositions]
#hitters['group'] = group_players(hitters.drop(['AVG rank', 'SB rank', 'RBI rank', 'HR rank', 'R rank', 'Razzball_Rank','Team','ESPN'], axis=1))
for i in range(len(groups)):
    groups[i] = groups[i].sort_values(by=['Ovr'],ascending=True)
    hitter_tables[HitterPositions[i]] = groups[i]
    #groups[i].to_csv('{}.csv'.format(HitterPositions[i]))

pitchers = get_pitchers().set_index('Name')
pitchers.sort_values(by=['Razzball_Rank'],inplace=True,ascending=True)

#groups = group_pitchers_pos(pitchers)
groups = [split_by_pos(pitchers, x) for x in PitcherPositions]
for i in range (len(groups)):
    groups[i] = groups[i].sort_values(by=['Ovr'],ascending=True)
    pitcher_tables[PitcherPositions[i]] = groups[i]
    
pitcher_tables['RP']


,WHIP rank,ERA rank,K rank,W rank,Razzball_Rank,Team,POS,W,SV,ERA,WHIP,K,Ovr,SV rank
Name,,,,,,,,,,,,,,
Josh Hader,2,2,0,NaN,192,MIL,RP,4.1,30,2.76,0.95,116.0,2.50,6.0
Edwin Diaz,1,1,1,NaN,234,NYM,RP,3.7,25,2.69,0.95,97.7,4.75,16.0
Kirby Yates,6,7,10,NaN,249,SD,RP,3.5,35,2.99,0.99,88.4,6.25,2.0
Nick Anderson,3,6,7,NaN,242,TB,RP,3.6,15,2.98,0.97,89.2,9.75,23.0
Aroldis Chapman,9,3,39,NaN,341,NYY,RP,3.0,35,2.80,1.02,74.4,13.00,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Trevor Kelley,434,459,413,NaN,635,PHI,RP,1.1,0,5.30,1.38,19.1,387.25,243.0
Kodi Medeiros,449,448,444,NaN,725,CHW,RP,0.4,0,4.86,1.43,9.0,387.75,210.0
Justin Lawrence,455,452,448,NaN,726,COL,RP,0.4,0,4.95,1.50,8.2,391.00,209.0


In [7]:
all_hitters = group_by_pos(get_hitters().set_index('Name'))[0].sort_values(by='Ovr',ascending=True).reset_index();
 
hitters


,AVG rank,SB rank,RBI rank,HR rank,R rank,Razzball_Rank,Team,ESPN,R,HR,RBI,SB,AVG,Ovr
Name,,,,,,,,,,,,,,
Mike Trout,7,42,9,1,4,19,LAA,OF,115.8,43.0,109.8,13.2,0.297,12.6
Cody Bellinger,29,47,1,3,1,31,LAD,1B,118.9,42.6,118.1,12.5,0.287,16.2
Cody Bellinger,29,47,1,3,1,31,LAD,OF,118.9,42.6,118.1,12.5,0.287,16.2
Christian Yelich,3,15,33,26,9,42,MIL,OF,112.2,34.7,96.5,21.4,0.304,17.2
Ronald Acuna Jr.,39,4,36,13,6,6,ATL,OF,113.7,36.9,94.8,29.4,0.283,19.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ronnie Dawson,674,631,676,662,673,678,HOU,OF,1.3,0.3,1.2,0.3,0.203,663.2
Christian Kelley,667,669,667,663,672,658,PIT,C,1.4,0.3,1.4,0.1,0.207,667.6
Ian Rice,656,666,675,673,675,673,CHC,C,1.2,0.3,1.2,0.1,0.209,669.0


In [116]:
def minimization_fun(x):
    sm = 0
    count = 0
    for key in hitter_tables:
        for metric in HitterMetrics:
            sm = sm + hitter_tables[key].iloc[int(x[count])].loc[metric]
        count = count + 1
    for metric in HitterMetrics:
        sm = sm + hitter_tables['util'].iloc[int(x[count])].loc[metric]
    count = count + 1
    for key in range(4):
        for metric in HitterMetrics:
            sm = sm + hitter_tables['OF'].iloc[int(x[count])].loc[metric]
        count = count + 1
    for key in range(8):
        for metric in PitcherMetrics:
            sm = sm + pitcher_tables['SP'].iloc[int(x[count])].loc[metric]
        count = count + 1
    for key in range(3):
        for metric in PitcherMetrics:
            sm = sm + pitcher_tables['RP'].iloc[int(x[count])].loc[metric]
        count = count + 1
    return sm

#minimization_fun([10]*25)
# need to add constraint, need to make minimize only check discrete values
#res = optimize.minimize(minimization_fun, x0=[10]*25, method = 'Nelder-Mead', options={'maxiter':10000})
#res.x

def find_min():
    # 20 1B, 20 2B, 20 SS, 20 3B
    # 70 OF, 100 SP, 45 RP
    # = 50,400,000,000 iterations
    
    # 18 1B, 18 2B, 18 SS, 18 3B
    # 70 OF, 80 SP, 40 RP
    # = 23,514,624,000 iterations
    prob = LpProblem("IdealRosterProblem",LpMinimize)
    
    all_players = get_hitter_prices(get_hitters()).set_index('Name').nsmallest(200,'Ovr').append(get_pitcher_prices(get_pitchers()).set_index('Name').nsmallest(200,'Ovr'))
    all_players.fillna(0, inplace=True)
    all_names = list(all_players.index)
    print(all_players['$'])
    allCosts = dict(zip(all_names,all_players['$']))
    
    allRuns = dict(zip(all_names,all_players['R']))
    allHRs = dict(zip(all_names,all_players['HR']))
    allRBIs = dict(zip(all_names,all_players['RBI']))
    allSBs = dict(zip(all_names,all_players['SB']))
    allAVG = dict(zip(all_names,all_players['AVG']))
    
    allWs = dict(zip(all_names,all_players['W']))
    allKs = dict(zip(all_names,all_players['K']))
    allSVs = dict(zip(all_names,all_players['SV']))
    allERA = dict(zip(all_names,all_players['ERA']))
    allWHIP = dict(zip(all_names,all_players['WHIP']))
    allPOS = dict(zip(all_names,all_players['POS']))
    
#     # C
#     cNames = list(hitter_tables['C']['Name'])
#     cCosts = dict(zip(hitter_tables['C']['$']))
#     cRuns = dict(zip(hitter_tables['C']['R']))
#     cHRs = dict(zip(hitter_tables['C']['HR']))
#     cRBIs = dict(zip(hitter_tables['C']['RBI']))
#     cSBs = dict(zip(hitter_tables['C']['SB']))
#     cAVG = dict(zip(hitter_tables['C']['AVG']))
    
#     # 1B
#     fstNames = list(hitter_tables['1B']['Name'])
#     fstCosts = dict(zip(hitter_tables['1B']['$']))
#     fstRuns = dict(zip(hitter_tables['1B']['R']))
#     fstHRs = dict(zip(hitter_tables['1B']['HR']))
#     fstRBIs = dict(zip(hitter_tables['1B']['RBI']))
#     fstSBs = dict(zip(hitter_tables['1B']['SB']))
#     fstAVG = dict(zip(hitter_tables['1B']['AVG']))
    
#     # 2B
#     sndNames = list(hitter_tables['2B']['Name'])
#     sndCosts = dict(zip(hitter_tables['2B']['$']))
#     sndRuns = dict(zip(hitter_tables['2B']['R']))
#     sndHRs = dict(zip(hitter_tables['2B']['HR']))
#     sndRBIs = dict(zip(hitter_tables['2B']['RBI']))
#     sndSBs = dict(zip(hitter_tables['2B']['SB']))
#     sndAVG = dict(zip(hitter_tables['2B']['AVG']))
    
#     # 3B
#     trdNames = list(hitter_tables['3B']['Name'])
#     trdCosts = dict(zip(hitter_tables['3B']['$']))
#     trdRuns = dict(zip(hitter_tables['3B']['R']))
#     trdHRs = dict(zip(hitter_tables['3B']['HR']))
#     trdRBIs = dict(zip(hitter_tables['3B']['RBI']))
#     trdSBs = dict(zip(hitter_tables['3B']['SB']))
#     trdAVG = dict(zip(hitter_tables['3B']['AVG']))
    
#     # SS
#     ssNames = list(hitter_tables['SS']['Name'])
#     ssCosts = dict(zip(hitter_tables['SS']['$']))
#     ssRuns = dict(zip(hitter_tables['SS']['R']))
#     ssHRs = dict(zip(hitter_tables['SS']['HR']))
#     ssRBIs = dict(zip(hitter_tables['SS']['RBI']))
#     ssSBs = dict(zip(hitter_tables['SS']['SB']))
#     ssAVG = dict(zip(hitter_tables['SS']['AVG']))
    
#     # OF 
#     ofNames = list(hitter_tables['OF']['Name'])
#     ofCosts = dict(zip(hitter_tables['OF']['$']))
#     ofRuns = dict(zip(hitter_tables['OF']['R']))
#     ofHRs = dict(zip(hitter_tables['OF']['HR']))
#     ofRBIs = dict(zip(hitter_tables['OF']['RBI']))
#     ofSBs = dict(zip(hitter_tables['OF']['SB']))
#     ofAVG = dict(zip(hitter_tables['OF']['AVG']))
    
#     # SP
#     spNames = list(pitcher_tables['SP']['Name'])
#     spCosts = dict(zip(pitcher_tables['SP']['$']))
#     spWs = dict(zip(pitcher_tables['SP']['W']))
#     spKs = dict(zip(pitcher_tables['SP']['K']))
#     spSVs = dict(zip(pitcher_tables['SP']['SV']))
#     spERA = dict(zip(pitcher_tables['SP']['ERA']))
#     spWHIP = dict(zip(pitcher_tables['SP']['WHIP']))
    
#     # RP
#     rpNames = list(pitcher_tables['RP']['Name'])
#     rpCosts = dict(zip(pitcher_tables['RP']['$']))
#     rpWs = dict(zip(pitcher_tables['RP']['W']))
#     rpKs = dict(zip(pitcher_tables['RP']['K']))
#     rpSVs = dict(zip(pitcher_tables['RP']['SV']))
#     rpERA = dict(zip(pitcher_tables['RP']['ERA']))
#     rpWHIP = dict(zip(pitcher_tables['RP']['WHIP']))
    
    player_vars = LpVariable.dicts("Players",all_names,lowBound=0,upBound=1,cat='Integer')
    
    prob += lpSum([(allRuns[i]+allHRs[i]+allRBIs[i]+allSBs[i]+allAVG[i]+allWs[i]+allKs[i]+allSVs[i]+allERA[i]+allWHIP[i])*player_vars[i] for i in all_names])
    
    prob += lpSum([allCosts[i]*player_vars[i] for i in all_names]) <= 200
    
    prob += lpSum([(allPOS[i]=='C')*player_vars[i] for i in all_names]) == 1
    
    # update these based on position depth
    prob += lpSum([(allPOS[i]=='1B')*player_vars[i] for i in all_names]) == 1
    prob += lpSum([(allPOS[i]=='2B')*player_vars[i] for i in all_names]) == 2
    prob += lpSum([(allPOS[i]=='3B')*player_vars[i] for i in all_names]) == 2
    prob += lpSum([(allPOS[i]=='SS')*player_vars[i] for i in all_names]) == 2
    
    prob += lpSum([(allPOS[i]=='OF')*player_vars[i] for i in all_names]) == 6
    
    prob += lpSum([(allPOS[i]=='SP')*player_vars[i] for i in all_names]) == 8
    
    prob += lpSum([(allPOS[i]=='RP')*player_vars[i] for i in all_names]) == 3
    
    prob.solve()
    
    print("Status:", LpStatus[prob.status])
    for v in prob.variables():
        if v.varValue>0:
            print(v.name, "=", v.varValue)
            
find_min()

Name
Mike Trout          37.8
Cody Bellinger      37.9
Cody Bellinger      37.9
Christian Yelich    38.3
Ronald Acuna Jr.    39.7
                    ... 
David Robertson    -10.8
Brad Peacock        -6.4
Brad Keller         -6.4
Kevin Ginkel        -2.3
Wade Miley          -5.6
Name: $, Length: 400, dtype: float64
Status: Optimal
Players_Alex_Gordon = 1.0
Players_Andrelton_Simmons = 1.0
Players_Austin_Hays = 1.0
Players_Austin_Voth = 1.0
Players_Brent_Honeywell = 1.0
Players_Bryse_Wilson = 1.0
Players_Daniel_Murphy = 1.0
Players_Dustin_May = 1.0
Players_Garrett_Hampson = 1.0
Players_Jimmy_Nelson = 1.0
Players_Jonathan_Loaisiga = 1.0
Players_Jordan_Hicks = 1.0
Players_Kwang_Hyun_Kim = 1.0
Players_Luis_Arraez = 1.0
Players_Miguel_Rojas = 1.0
Players_Nick_Burdi = 1.0
Players_Robbie_Erlin = 1.0
Players_Sam_Selman = 1.0
Players_Shogo_Akiyama = 1.0
Players_Victor_Reyes = 1.0
Players_Willson_Contreras = 1.0
Players_Yandy_Diaz = 1.0


In [23]:
all_players = get_hitters().set_index('Name').nsmallest(200,'Ovr').append(get_pitchers().set_index('Name').nsmallest(180,'Ovr'))
all_players.sort_values(by=['Ovr'], inplace=True)
all_players.drop(['AVG','R','RBI','HR','SB','WHIP','ERA','W','SV','K'], axis=1, inplace=True)
all_players

,AVG rank,SB rank,RBI rank,HR rank,R rank,Razzball_Rank,Team,Ovr,POS,WHIP rank,ERA rank,K rank,SV rank,W rank
Name,,,,,,,,,,,,,,
Mike Trout,7.0,42.0,9.0,1.0,4.0,19,LAA,12.6,OF,NaN,NaN,NaN,NaN,NaN
Cody Bellinger,29.0,47.0,1.0,3.0,1.0,31,LAD,16.2,1B,NaN,NaN,NaN,NaN,NaN
Cody Bellinger,29.0,47.0,1.0,3.0,1.0,31,LAD,16.2,OF,NaN,NaN,NaN,NaN,NaN
Christian Yelich,3.0,15.0,33.0,26.0,9.0,42,MIL,17.2,OF,NaN,NaN,NaN,NaN,NaN
Ronald Acuna Jr.,39.0,4.0,36.0,13.0,6.0,6,ATL,19.6,OF,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Brad Wieck,NaN,NaN,NaN,NaN,NaN,395,CHC,285.2,RP,59.0,71.0,285.0,646.0,365.0
Collin McHugh,NaN,NaN,NaN,NaN,NaN,188,FA,285.2,RP,150.0,191.0,196.0,699.0,190.0
Amir Garrett,NaN,NaN,NaN,NaN,NaN,230,CIN,285.4,RP,166.0,109.0,174.0,730.0,248.0
